# Reading Signup Genius

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


# %%
import requests 
import pandas as pd
import time 
from datetime import datetime
import numpy as np
import re

print("Reading Timetree website and analyzing data...")

# Reading in Signup Genius URL and data

intl_url = 'https://www.signupgenius.com/go/10c0f4aaeab2fa2fac07-july1'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(intl_url, headers=header)

# Finding the right table on the page

dfs = pd.read_html(r.text)
df = dfs[3]
df.rename(columns={0:'Role', 2:'Person'} , inplace = True)
df.drop(columns=1, inplace = True)

# %%
df.head(25)

Reading Timetree website and analyzing data...


,Role,Person
0,Sunday Greeter (3) (indicate if you have or h...,2 of 3 slots filled Jackie Barber Jac Cottam ...
1,Sunday Secretary,Sign Up
2,Sunday Topic Bringer,Ricki Gage Asky Basky
3,Monday Greeter (3) (indicate if you have or h...,1 of 3 slots filled Joshua Reeves Sign Up
4,Monday Secretary,JoAnna Saunders
5,Monday Topic Bringer,Joshua Reeves
6,Tuesday Greeter (3) (indicate if you have or ...,2 of 3 slots filled Joshua Reeves Jackie Barb...
7,Tuesday Secretary,Jackie Barber
8,Tuesday Topic Bringer,JoAnna Saunders
9,Wednesday Greeter (3) (indicate if you have o...,2 of 3 slots filled Ricki Gage JoAnna Saunder...


In [2]:

to_remove = {
    '2 of 3 slots filled',
    '1 of 3 slots filled',
    'All slots filled',
    'Sign Up',
    'O yeah',
    'Asky Basky'
}

to_replace = {
    'Joshua Reeves Reeves' : 'Joshua Reeves',
    'Bill Collins Collins' : 'Bill Collins'
}

df['Person'] = df['Person'].str.replace('|'.join(to_remove),"", regex = True)
df['Person'] = df['Person'].str.replace('Joshua Reeves Reeves','Joshua Reeves')
df['Person'] = df['Person'].str.replace('Bill Collins Collins','Bill Collins')
df['Person'] = df['Person'].str.replace('Jesse J Gage','Jesse Gage')

df['Role'] = df['Role'].str.split('(').str[0]

# %%
# Adding emoji for data:attributes:title

emoji = []


for i in df['Role']: 
    if 'Greeter' in i:
        i = "\U0001F366 " # soft ice cream
    elif 'Topic' in i: 
        i = "\U0001F367 " # shaved ice
    elif 'Secretary' in i: 
        i = "\U0001F9C9 " # coconut
    emoji.append(i)

df['Emoji'] = emoji

# %%
# Adding verb for data:attributes:title

verb = []

for i in df['Role']: 
    if 'Greeter' in i:
        i = 'greet!'
    elif 'Topic' in i: 
        i = 'brings topic!'
    elif 'Secretary' in i: 
        i = 'Secretarys!'
    verb.append(i)

df['Verb'] = verb
df.head(40)

df['first'] = df['Person'].str.split().str[0]
df['second'] = df['Person'].str.split().str[2]
df['third'] = df['Person'].str.split().str[4]
df = df.fillna('')
df['Combined'] = df['first'] + " " + df['second'] + " " + df['third']
df['Count'] = df['Combined'].str.split().str.len()
df.head(50)

,Role,Person,Emoji,Verb,first,second,third,Combined,Count
0,Sunday Greeter,Jackie Barber Jac Cottam,🍦,greet!,Jackie,Jac,,Jackie Jac,2
1,Sunday Secretary,,🧉,Secretarys!,,,,,0
2,Sunday Topic Bringer,Ricki Gage,🍧,brings topic!,Ricki,,,Ricki,1
3,Monday Greeter,Joshua Reeves,🍦,greet!,Joshua,,,Joshua,1
4,Monday Secretary,JoAnna Saunders,🧉,Secretarys!,JoAnna,,,JoAnna,1
5,Monday Topic Bringer,Joshua Reeves,🍧,brings topic!,Joshua,,,Joshua,1
6,Tuesday Greeter,Joshua Reeves Jackie Barber,🍦,greet!,Joshua,Jackie,,Joshua Jackie,2
7,Tuesday Secretary,Jackie Barber,🧉,Secretarys!,Jackie,,,Jackie,1
8,Tuesday Topic Bringer,JoAnna Saunders,🍧,brings topic!,JoAnna,,,JoAnna,1
9,Wednesday Greeter,Ricki Gage JoAnna Saunders,🍦,greet!,Ricki,JoAnna,,Ricki JoAnna,2


In [3]:

df.loc[df['Count'] == 3, 'data:attributes:title'] = df['Emoji'] + df['first'] + ", " + df['second'] + " & " + df['third'] + " " + df['Verb']
df.loc[df['Count'] == 2, 'data:attributes:title'] = df['Emoji'] + df['first'] + " & " + df['second'] + " " + df['Verb']
df.loc[df['Count'] == 1, 'data:attributes:title'] = df['Emoji'] + df['first'] + " " +  df['Verb']

# %%
# Cleaning for export 

df['Day'] = df['Role'].str.split().str[0]
df['Role'] = df['Role'].str.split(" ", 1).str[1]
df.drop(columns={'Person','Verb','first','second','third','Combined','Count','Emoji'} , inplace = True)
df = df[['Day','Role','data:attributes:title']]
df.head()


# In[6]:


script_url = 'https://docs.google.com/document/d/1tVsWegpQ8LR7J20m8sRVA9z-XuKPzDwX7d8yQER-X1c/edit?usp=sharing'
zoom_url = 'https://us04web.zoom.us/j/406536344'
df.head()

url = []

for i in df['Role']: 
    if 'Greeter' in i:
        i = zoom_url
    elif 'Topic' in i: 
        i = zoom_url
    elif 'Secretary' in i: 
        i = script_url
    url.append(i)


df['url'] = url
df['all_day'] = 'false'
df['type'] = 'event'
df['description'] = ''


df = df.fillna('')
df.head(40)

,Day,Role,data:attributes:title,url,all_day,start_at,end_at,type,description
0,Sunday,Greeter,🍦 Jackie & Jac greet!,https://us04web.zoom.us/j/406536344,false,,,event,
1,Sunday,Secretary,,https://docs.google.com/document/d/1tVsWegpQ8L...,false,,,event,
2,Sunday,Topic Bringer,🍧 Ricki brings topic!,https://us04web.zoom.us/j/406536344,false,,,event,
3,Monday,Greeter,🍦 Joshua greet!,https://us04web.zoom.us/j/406536344,false,,,event,
4,Monday,Secretary,🧉 JoAnna Secretarys!,https://docs.google.com/document/d/1tVsWegpQ8L...,false,,,event,
5,Monday,Topic Bringer,🍧 Joshua brings topic!,https://us04web.zoom.us/j/406536344,false,,,event,
6,Tuesday,Greeter,🍦 Joshua & Jackie greet!,https://us04web.zoom.us/j/406536344,false,,,event,
7,Tuesday,Secretary,🧉 Jackie Secretarys!,https://docs.google.com/document/d/1tVsWegpQ8L...,false,,,event,
8,Tuesday,Topic Bringer,🍧 JoAnna brings topic!,https://us04web.zoom.us/j/406536344,false,,,event,
9,Wednesday,Greeter,🍦 Ricki & JoAnna greet!,https://us04web.zoom.us/j/406536344,false,,,event,


In [34]:
start_time = []

for i in df['Role']: 
    if 'Sec' in i:
        i =  '22:14'
    elif 'Topic' in i: 
        i = '22:30'
    else: 
        i = '22:37'
    start_time.append(i)

for i in start_time: 
    datetime.strptime(i,'%H:%M')
start_time.append(i)


df.head(40)

,Day,Role,data:attributes:title,url,all_day,start_at,end_at,type,description,start_time
0,Sunday,Greeter,🍦 Jackie & Jac greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,22:37
1,Sunday,Secretary,,https://docs.google.com/document/d/1tVsWegpQ8L...,false,1900-01-01 22:15:00,,event,,22:14
2,Sunday,Topic Bringer,🍧 Ricki brings topic!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,22:30
3,Monday,Greeter,🍦 Joshua greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,22:37
4,Monday,Secretary,🧉 JoAnna Secretarys!,https://docs.google.com/document/d/1tVsWegpQ8L...,false,1900-01-01 22:15:00,,event,,22:14
5,Monday,Topic Bringer,🍧 Joshua brings topic!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,22:30
6,Tuesday,Greeter,🍦 Joshua & Jackie greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,22:37
7,Tuesday,Secretary,🧉 Jackie Secretarys!,https://docs.google.com/document/d/1tVsWegpQ8L...,false,1900-01-01 22:15:00,,event,,22:14
8,Tuesday,Topic Bringer,🍧 JoAnna brings topic!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,22:30
9,Wednesday,Greeter,🍦 Ricki & JoAnna greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,22:37


# Export File to CSV for Load

In [5]:
# print("Data exporting...")
# df.to_csv(r'/Users/jaccottam/Desktop/Projects/INTL/db/timetree.csv')
# print("Data exported to db Check for file named timetree.csv.")

# Creating Date Table for Meeting Population

Use the following table to create dates for the meeting in the future.

In [6]:
def create_date_table(start='2022-08-01', end='2022-08-31'):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df["month"] = df.Date.dt.month
    df["day_of_Month"] = df.Date.dt.day
    df["year"] = df.Date.dt.year
    df["Day"] = df.Date.dt.day_name()
    return df

dates = create_date_table()

dates.head(50)

,Date,month,day_of_Month,year,Day
0,2022-08-01,8,1,2022,Monday
1,2022-08-02,8,2,2022,Tuesday
2,2022-08-03,8,3,2022,Wednesday
3,2022-08-04,8,4,2022,Thursday
4,2022-08-05,8,5,2022,Friday
5,2022-08-06,8,6,2022,Saturday
6,2022-08-07,8,7,2022,Sunday
7,2022-08-08,8,8,2022,Monday
8,2022-08-09,8,9,2022,Tuesday
9,2022-08-10,8,10,2022,Wednesday


# Working on Functions

In [7]:
df.head(25)

def splitz(x):
    y = x.str.lower()
    return y.str.split()


df.head(45)

,Day,Role,data:attributes:title,url,all_day,start_at,end_at,type,description
0,Sunday,Greeter,🍦 Jackie & Jac greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,
1,Sunday,Secretary,,https://docs.google.com/document/d/1tVsWegpQ8L...,false,1900-01-01 22:15:00,,event,
2,Sunday,Topic Bringer,🍧 Ricki brings topic!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,
3,Monday,Greeter,🍦 Joshua greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,
4,Monday,Secretary,🧉 JoAnna Secretarys!,https://docs.google.com/document/d/1tVsWegpQ8L...,false,1900-01-01 22:15:00,,event,
5,Monday,Topic Bringer,🍧 Joshua brings topic!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,
6,Tuesday,Greeter,🍦 Joshua & Jackie greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,
7,Tuesday,Secretary,🧉 Jackie Secretarys!,https://docs.google.com/document/d/1tVsWegpQ8L...,false,1900-01-01 22:15:00,,event,
8,Tuesday,Topic Bringer,🍧 JoAnna brings topic!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,
9,Wednesday,Greeter,🍦 Ricki & JoAnna greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,


In [8]:
write_df = pd.merge(df,dates,how = 'inner')
write_df.head(100)

,Day,Role,data:attributes:title,url,all_day,start_at,end_at,type,description,Date,month,day_of_Month,year
0,Sunday,Greeter,🍦 Jackie & Jac greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,2022-08-07,8,7,2022
1,Sunday,Greeter,🍦 Jackie & Jac greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,2022-08-14,8,14,2022
2,Sunday,Greeter,🍦 Jackie & Jac greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,2022-08-21,8,21,2022
3,Sunday,Greeter,🍦 Jackie & Jac greet!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,2022-08-28,8,28,2022
4,Sunday,Secretary,,https://docs.google.com/document/d/1tVsWegpQ8L...,false,1900-01-01 22:15:00,,event,,2022-08-07,8,7,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Saturday,Secretary,🧉 Jac Secretarys!,https://docs.google.com/document/d/1tVsWegpQ8L...,false,1900-01-01 22:15:00,,event,,2022-08-27,8,27,2022
84,Saturday,Topic Bringer,🍧 Dana brings topic!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,2022-08-06,8,6,2022
85,Saturday,Topic Bringer,🍧 Dana brings topic!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,2022-08-13,8,13,2022
86,Saturday,Topic Bringer,🍧 Dana brings topic!,https://us04web.zoom.us/j/406536344,false,1900-01-01 22:15:00,,event,,2022-08-20,8,20,2022
